# Description

Data dictnionary

- id : o id do documento no Oasisbr
- oai_identifier_str : id externo que vem do repositório de origem
- instname_str : nome da instituição de origem
- instacron_str : acrônimo da instituição de origem
- network_acronym_str : acrônimo da rede de origem (pode descartar)
- network_name_str : nome da rede origem (pode descartar)
- title : título
- author : autores
- publishDate  : data da publicação
- format : tipo de documento (article, masterThesis, doctoralThesis, …)
- description : abstract
- topic : keywords
- language : idioma do documento
- eu_rights_str_mv : tipo de acesso (openAccess, closed, …)
- url : url externa para o documento

In [133]:
# general
import sys
import os
import logging
import time

# read json
from zipfile import ZipFile
import ijson #ijson-3.1.4
import pandas as pd
# web3
import requests
import json

from hexbytes import HexBytes
from web3 import Web3

# SCRIPT_DIR = os.path.dirname(os.path.abspath('__file__'))
PROJECT_DIR = os.path.dirname(os.path.abspath(''))
sys.path.append(os.path.dirname(PROJECT_DIR))

from dARK.util import setup
from dARK.util.libs import invoke_contract_experimental

# Experiment

## setup

In [49]:
# logging.basicConfig(filename='myapp.log', level=logging.INFO)
# logging.basicConfig(level=logging.INFO)

input file

In [50]:
infile = '/mnt/d/ibict/oasisbr/oasisbr_1.zip'

bc

In [51]:
addr = '0xf17f52151EbEF6C7334FAD080c5704D77216b732'  #from setup
auth_id = '0x47be0ba38f0ab57d6469dedd62c007b4605d000efe0ceeade9696543076c59bc'
auth_id = Web3.toBytes(HexBytes(auth_id))
# import ujson #ujson-5.6.0
# with open(infile) as f:
#     file_contents = ujson.load(f)

##  metoodo 0

In [52]:
w3 = setup.load_blockchain_driver()
deployed_contracts = setup.load_deployed_smart_contracts(w3)
dpid_db = deployed_contracts['PidDB.sol']
epid_db = deployed_contracts['ExternalPidDB.sol']
auth_db = deployed_contracts['AuthoritiesDB.sol']

dpid_service = deployed_contracts['PIDService.sol']
epid_service = deployed_contracts['ExternalPIDService.sol']
sets_service = deployed_contracts['SearchTermService.sol']
auth_service = deployed_contracts['AuthoritiesService.sol']


chain_id,min_gas_price,pk = setup.get_exec_parameters()
account = w3.eth.account.privateKeyToAccount(pk)

/mnt/d/ibict/workspace/dARK_limits/dark/config.ini


In [139]:
print("DNMA number :: " + str(auth_db.caller.count_dnma()))
print("\t DNMA :: " + Web3.toHex(auth_db.caller.get_dnma_by_index(0) ) ) # converte o indice 0 

msg = "   DecentralizedNameMappingAuthority :: {} [ ror={} , shoulder_prefix={} ]".format(auth_db.caller.get_dnma(auth_id)[-1],
                                                                                        auth_db.caller.get_dnma(auth_id)[1],
                                                                                        auth_db.caller.get_dnma(auth_id)[2])

msg

DNMA number :: 1
	 DNMA :: 0x47be0ba38f0ab57d6469dedd62c007b4605d000efe0ceeade9696543076c59bc


'   DecentralizedNameMappingAuthority :: 0xf17f52151EbEF6C7334FAD080c5704D77216b732 [ ror=3wf3 , shoulder_prefix=fkwf ]'

In [141]:
def read_metadata(record):
    #reading metadata
    title = record['title']
    
    try:
        authors = record['author']
    except KeyError:
        authors = []
    try:
        topics = record['topic']
    except KeyError:
        topics = []
    try:
        url = record['url'][0]
    except KeyError:
        url = ""
    try:
        publishDate = record['publishDate'][0]
    except KeyError:
        publishDate = ""
    try:
        format = record['format'][0]
    except KeyError:
        format = ""

    return title, authors, topics, publishDate , url, publishDate, format

def create_payload(title, authors, publishDate, format):
    # pl_authors = ""
    # for i in range(len(authors)):
    #     if i > 0:
    #         pl_authors += ";" + authors[i]
    #     else:
    #         pl_authors += authors[i]
                
    payload = {
                            "title": title,
                            "authors": str(authors),
                            "format": format,
                            "publishDate" : publishDate
                }
    
    # if len(authors) > 1:
    # print(authors)
    
    return payload

In [142]:
sel = []
count = 0

# record['id'],payload_size, num_authors, num_topics, action , gas , time
experimental_data = []

with ZipFile(infile, 'r') as archive:
    file = archive.namelist()[0]
    with archive.open(file) as f:
        for record in ijson.items(f, "response.docs.item"):
            try:
                # if record['instacron_str'] == "UEPB":
                #     sel.append(record)

                title, authors, topics, publishDate , url, publishDate, format = read_metadata(record)
                
                payload = create_payload(title, authors, publishDate, format)
                payload_size = sys.getsizeof(str(payload))
                num_authors = len(authors)
                num_topics = len(topics)

                base_row = [record['id'],payload_size, num_authors, num_topics]

                #assing id
                # this is a critical operation mus be fast
                start_time = time.time()
                r_tx, gas = invoke_contract_experimental(w3,account,chain_id, dpid_service , 'assingID', addr)
                dark_id = r_tx['logs'][0]['topics'][1]
                delta = time.time() - start_time
                
                experimental_row = base_row.copy()
                experimental_row.append('assingID') #action
                experimental_row.append(gas)        #gas
                experimental_row.append(delta)      #detla time
                experimental_data.append(experimental_row)

                #addSearchTerm
                # assyncronous operation
                for st in topics:
                    start_time = time.time()
                    r_tx, gas = invoke_contract_experimental(w3,account,chain_id, dpid_service, 'addSearchTerm',dark_id, st)
                    delta = time.time() - start_time
                    experimental_row = base_row.copy()
                    experimental_row.append('addSearchTerm') #action
                    experimental_row.append(gas)        #gas
                    experimental_row.append(delta)      #detla time
                    experimental_data.append(experimental_row)
                
                #addUrl
                try:
                    if len(url) > 4:
                        start_time = time.time()
                        r_tx, est_gas = invoke_contract_experimental(w3,account,chain_id, dpid_service, 'add_externalLinks',dark_id, url) 
                        delta = time.time() - start_time
                        experimental_row = base_row.copy()
                        experimental_row.append('addExternalLink') #action
                        experimental_row.append(gas)        #gas
                        experimental_row.append(delta)      #detla time
                        experimental_data.append(experimental_row)
                except:
                    logging.error("record with with duplicated url id=[{}]".format(record['id']))
                

                #set_payload
                start_time = time.time()
                r_tx, est_gas = invoke_contract_experimental(w3,account,chain_id, dpid_service, 'set_payload',dark_id, str(payload))
                delta = time.time() - start_time
                
                experimental_row = base_row.copy()
                experimental_row.append('setPayload') #action
                experimental_row.append(gas)        #gas
                experimental_row.append(delta)      #detla time
                experimental_data.append(experimental_row)                
                # r_tx, est_gas = invoke_contract_experimental(w3,account,chain_id, dpid_service, 'set_payload',dark_id, str(payload_data)) 
                

                

            except KeyError:
                logging.error("record with no title id=[{}]".format(record['id']))
            # except:
            #     pass
            
            if count > 5:
                break
            count+=1


In [144]:
# sel[20:30]
# recors
# authors[0]
# url
df = pd.DataFrame(experimental_data)
df.columns = [ 'id','payload_len', 'num_authors', 'num_topics', 'action' , 'gas' , 'time']

DataFrame attributes
- **id** : oasis id
- **payload_len**: payload length iin bytes
- **num_authors** : number of authors
- **num_topics** : number of topics (search keys)
- **action** : actioin  performed
  - assingID : Assing a ark PID to an object
  - addSearchTerm : index the PID by a search term (topic)
  - addExternalLink: add link to the object
  - setPayload : set the payload
- **gas** : gas spend to execute the action
- **time** : time to execute the action

In [150]:
df.head(5)

,id,payload_len,num_authors,num_topics,action,gas,time
0,SBFPO-1_2fc1e8de0c870ff98b611a99775aa6ec,270,1,5,assingID,288587,2.039148
1,SBFPO-1_2fc1e8de0c870ff98b611a99775aa6ec,270,1,5,addSearchTerm,304586,2.392351
2,SBFPO-1_2fc1e8de0c870ff98b611a99775aa6ec,270,1,5,addSearchTerm,287043,5.440978
3,SBFPO-1_2fc1e8de0c870ff98b611a99775aa6ec,270,1,5,addSearchTerm,296184,1.996337
4,SBFPO-1_2fc1e8de0c870ff98b611a99775aa6ec,270,1,5,addSearchTerm,288825,2.013809


In [156]:
df.groupby('id').sum().time.mean()

23.12476352282933